<a href="https://colab.research.google.com/github/worachot-n/Text_summarization_T5/blob/main/1_4_Text_Summarization_Custom_Rouge_BertScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Kaggle')

# Install Library

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]
!pip install datasets rouge-score nltk
!pip install tensorboard

In [ ]:
!pip install bert-score

# Import Library

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules 
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

from datasets import load_dataset, load_metric

In [ ]:
%matplotlib inline

In [ ]:
metric_1 = load_metric("rouge")
metric_2 = load_metric("bertscore")

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Kaggle/BBC_News_Summary_with_filename.csv")

In [ ]:
df.sample(10)

,Unnamed: 0,articles,summaries,categories,articles_file,summaries_file
2060,2060,Microsoft takes on desktop search\n\nMicrosoft...,Search giant Google launched its desktop searc...,tech,237.txt,237.txt
417,417,US firm 'bids for Lacroix label'\n\nA US firm ...,A US firm has said it is in final negotiations...,business,418.txt,418.txt
1161,1161,Peers debate Crown succession law\n\nPeers are...,The Fabian Society's Sundar Katwala said the c...,politics,266.txt,266.txt
808,808,Fry set for role in Hitchhiker's\n\nActor Step...,"Fry will provide the voice of The Guide, an el...",entertainment,299.txt,299.txt
892,892,French honour for director Parker\n\nBritish f...,"""You have explored the possibilities of film w...",entertainment,383.txt,383.txt
1711,1711,Farrell saga to drag on - Lindsay\n\nWigan cha...,Although the move would be a lucrative one for...,sport,399.txt,399.txt
1617,1617,Kirwan demands Italy consistency\n\nItaly coac...,Italy coach John Kirwan has challenged his sid...,sport,305.txt,305.txt
1149,1149,Could rivalry overshadow election?\n\nTony Bla...,Lord Haskins even suggested Mr Blair should re...,politics,254.txt,254.txt
870,870,Super Size Me wins writers' award\n\nSuper Siz...,Spurlock was given his award on the same day t...,entertainment,361.txt,361.txt
97,97,IMF 'cuts' German growth estimate\n\nThe Inter...,The IMF will also reduce its growth estimate f...,business,098.txt,098.txt


In [ ]:
df_business = df.loc[df['categories'] == 'business']

In [ ]:
df_business = df_business[['articles', 'summaries']]
df_business

,articles,summaries
0,Ad sales boost Time Warner profit\n\nQuarterly...,TimeWarner said fourth quarter sales rose 2% t...
1,Dollar gains on Greenspan speech\n\nThe dollar...,The dollar has hit its highest level against t...
2,Yukos unit buyer faces loan claim\n\nThe owner...,Yukos' owner Menatep Group says it will ask Ro...
3,High fuel prices hit BA's profits\n\nBritish A...,"Rod Eddington, BA's chief executive, said the ..."
4,Pernod takeover talk lifts Domecq\n\nShares in...,Pernod has reduced the debt it took on to fund...
...,...,...
505,Trial begins of Spain's top banker\n\nThe tria...,Both executives helped Mr Botin orchestrate Sp...
506,UK economy ends year with spurt\n\nThe UK econ...,"Simon Rubinsohn, chief economist at Gerrard, s..."
507,HealthSouth ex-boss goes on trial\n\nThe forme...,Several former HealthSouth employees have alre...
508,Euro firms miss out on optimism\n\nMore than 9...,"Possibly as a result, the worry about low-cost..."


In [ ]:
df_business['articles'][0]

'Ad sales boost Time Warner profit\n\nQuarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.\n\nThe firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.\n\nTime Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to si

In [ ]:
df_business['summaries'][0]

"TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.Time Warner's fourth quarter profits were slightly better than analysts' expectations."

In [ ]:
df_business = df_business.replace(r'\n\n',' ', regex=True)
df_business

,articles,summaries
0,Ad sales boost Time Warner profit Quarterly pr...,TimeWarner said fourth quarter sales rose 2% t...
1,Dollar gains on Greenspan speech The dollar ha...,The dollar has hit its highest level against t...
2,Yukos unit buyer faces loan claim The owners o...,Yukos' owner Menatep Group says it will ask Ro...
3,High fuel prices hit BA's profits British Airw...,"Rod Eddington, BA's chief executive, said the ..."
4,Pernod takeover talk lifts Domecq Shares in UK...,Pernod has reduced the debt it took on to fund...
...,...,...
505,Trial begins of Spain's top banker The trial o...,Both executives helped Mr Botin orchestrate Sp...
506,UK economy ends year with spurt The UK economy...,"Simon Rubinsohn, chief economist at Gerrard, s..."
507,HealthSouth ex-boss goes on trial The former h...,Several former HealthSouth employees have alre...
508,Euro firms miss out on optimism More than 90% ...,"Possibly as a result, the worry about low-cost..."


In [ ]:
df_business["summaries"] = "summarize: "+df_business["summaries"]

In [ ]:
df_business.head()

,articles,summaries
0,Ad sales boost Time Warner profit Quarterly pr...,summarize: TimeWarner said fourth quarter sale...
1,Dollar gains on Greenspan speech The dollar ha...,summarize: The dollar has hit its highest leve...
2,Yukos unit buyer faces loan claim The owners o...,summarize: Yukos' owner Menatep Group says it ...
3,High fuel prices hit BA's profits British Airw...,"summarize: Rod Eddington, BA's chief executive..."
4,Pernod takeover talk lifts Domecq Shares in UK...,summarize: Pernod has reduced the debt it took...


In [ ]:
# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)

In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [ ]:
!pip install tensorboard

In [ ]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

In [ ]:
# SummaryWriter takes log directory as argument
writer = SummaryWriter()

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """
  running_loss = 0.0
  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)
      writer.add_scalar('loss', loss, epoch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if _%10==0:
      writer.add_scalar('training loss', running_loss / 10, epoch * len(loader) + _)
      running_loss = 0.0
  writer.close()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [ ]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text (article)
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text (summarize)
    "SEED": 42                     # set seed for reproducibility 

}

In [ ]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
      
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [ ]:
T5Trainer(dataframe=df_business[:10], source_text="articles", target_text="summaries", model_params=model_params, output_dir="./outputs/")

[11:22:36] [Model]: Loading t5-base...                     <ipython-input-26-b1eef7f6be59>:14
                                                                                             

[11:22:58] [Data]: Reading data...                         <ipython-input-26-b1eef7f6be59>:25
                                                                                             

                                         Sample Data                                         
+-------------------------------------------------------------------------------------------+
|                source_text                  |                 target_text                 |
|---------------------------------------------+---------------------------------------------|
|Ad sales boost Time Warner profit Quarterly  |  summarize: TimeWarner said fourth quarter  |
|profits at US media giant TimeWarner jumped  |  sales rose 2% to $11.1bn from $10.9bn.For  |
|76% to $1.13bn (Â£600m) for the three months | the full-year, TimeWarner posted a profit of|
| to December, from $639m year-earlier. The   |  $3.36bn, up 27% from its 2003 performance, |
|   firm, which is now one of the biggest     |         while revenues grew 6.4% to         |
|investors in Google, benefited from sales of | $42.09bn.Quarterly profits at US media giant|
| high-speed internet connections and higher  |  TimeWarner jumped 76% to $1.13bn (Â£600m)  |
|advert sales. TimeWarner said fourth quarter | for the three months to December, from $639m|
| sales rose 2% to $11.1bn from $10.9bn. Its  | year-earlier.However, the company said AOL's|
| profits were buoyed by one-off gains which  |  underlying profit before exceptional items |
|offset a profit dip at Warner Bros, and less |   rose 8% on the back of stronger internet  |
| users for AOL. Time Warner said on Friday   | advertising revenues.Its profits were buoyed|
|that it now owns 8% of search-engine Google. |  by one-off gains which offset a profit dip |
|But its own internet business, AOL, had has  |  at Warner Bros, and less users for AOL.For |
|mixed fortunes. It lost 464,000 subscribers  |   2005, TimeWarner is projecting operating  |
|  in the fourth quarter profits were lower   |    earnings growth of around 5%, and also   |
|   than in the preceding three quarters.     |   expects higher revenue and wider profit   |
| However, the company said AOL's underlying  |  margins.It lost 464,000 subscribers in the |
| profit before exceptional items rose 8% on  |  fourth quarter profits were lower than in  |
| the back of stronger internet advertising   |  the preceding three quarters.Time Warner's |
| revenues. It hopes to increase subscribers  | fourth quarter profits were slightly better |
|   by offering the online service free to    |         than analysts' expectations.        |
| TimeWarner internet customers and will try  |                                             |
|  to sign up AOL's existing customers for    |                                             |
|high-speed broadband. TimeWarner also has to |                                             |
| restate 2000 and 2003 results following a   |                                             |
|    probe by the US Securities Exchange      |                                             |
|    Commission (SEC), which is close to      |                                             |
|  concluding. Time Warner's fourth quarter   |                                             |
|profits were slightly better than analysts'  |                                             |
|  expectations. But its film division saw    |                                             |
|   profits slump 27% to $284m, helped by     |                                             |
| box-office flops Alexander and Catwoman, a  |                                             |
|  sharp contrast to year-earlier, when the   |                                             |
|  third and final film in the Lord of the    |                                             |
|   Rings trilogy boosted results. For the    |                                             |
|  full-year, TimeWarner posted a profit of   |                                             |
| $3.36bn, up 27% from its 2003 performance,  |                                             |
| while revenues grew 6.4% to $42.09bn. "Our  |     

FULL Dataset: (10, 2)

TRAIN Dataset: (8, 2)

...

Completed 0

[11:23:06] [Validation Completed.]                         <ipython-input-26-b1eef7f6be59>:95
                                                                                             

[Model] Model saved @ ./outputs/model_files

[Validation] Generation on Validation data saved @ ./outputs/predictions.csv

[Logs] Logs saved @ ./outputs/logs.txt

In [ ]:
metric_1

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

In [ ]:
metric_2

Metric(name: "bert_score", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
BERTScore Metrics with the hashcode from a source against one or more references.

Args:
    predictions (list of str): Prediction/candidate sentences.
    references (list of str or list of list of str): Reference sentences.
    lang (str): Language of the sentences; required (e.g. 'en').
    model_type (str): Bert specification, default using the suggested
        model for the target language; has to specify at least one of
        `model_type` or `lang`.
    num_layers (int): The layer of representation to use,
        default using the number of layers tuned on WMT16 correlation data.
    verbose (bool): Turn on intermediate status update.
    idf (bool or dict): Use idf weighting; can also be a precomputed idf_dict.
    device (str): On which the contextual embedding model will be a

In [ ]:
fake_preds = ["hello there", "general tokyo"]
fake_labels = ["hello there", "general kenobi"]
metric_1.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=0.5, recall=0.5, fmeasure=0.5), mid=Score(precision=0.75, recall=0.75, fmeasure=0.75), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.5, recall=0.5, fmeasure=0.5), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=0.5, recall=0.5, fmeasure=0.5), mid=Score(precision=0.75, recall=0.75, fmeasure=0.75), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=0.5, recall=0.5, fmeasure=0.5), mid=Score(precision=0.75, recall=0.75, fmeasure=0.75), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [ ]:
# fake_preds = ["hello there", "general tokyo"]
# fake_labels = ["hello there", "general kenobi"]
# metric_2.compute(predictions=fake_preds, references=fake_labels)

In [ ]:
eva = pd.read_csv('/content/drive/MyDrive/Kaggle/outputs/predictions.csv')

In [ ]:
from bert_score import BERTScorer

In [ ]:
eva

,Unnamed: 0,Generated Text,Actual Text
0,0,British Airways reported pre-tax profit of £75...,"summarize: Rod Eddington, BA's chief executive..."
1,1,jobs growth still slow in the US The US create...,summarize: The job gains mean that President B...


In [ ]:
eva['Generated Text'].to_list()

['British Airways reported pre-tax profit of £75m ($141m) for the three months to 31 December 2004 compared with £125m a year earlier. fuel costs rose by £106m or 47.3% in third quarter. but aviation analyst says surcharge revenues will still be way short of predicted extra £250m.',
 'jobs growth still slow in the US The US created fewer jobs than expected in January, but a fall in jobseekers pushed the unemployment rate to its lowest level in three years.']

In [ ]:
eva['Actual Text'].to_list()

['summarize: Rod Eddington, BA\'s chief executive, said the results were "respectable" in a third quarter when fuel costs rose by £106m or 47.3%.To help offset the increased price of aviation',
 'summarize: The job gains mean that President Bush can celebrate - albeit by a very fine margin - a net growth in jobs in the US economy in his first term in office.Analysts said the']

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
rouge.get_scores(eva['Generated Text'].to_list(), eva['Actual Text'].to_list(), avg=True)

{'rouge-1': {'f': 0.3173076874256414,
  'p': 0.29418985270049103,
  'r': 0.35860215053763445},
 'rouge-2': {'f': 0.13286712798150982,
  'p': 0.11736444749485243,
  'r': 0.1576036866359447},
 'rouge-l': {'f': 0.2637362588542129,
  'p': 0.23649754500818332,
  'r': 0.3086021505376344}}

In [ ]:
refs = eva['Actual Text'].to_list()

hyps = eva['Generated Text'].to_list()

scorer = BERTScorer(lang="en", rescale_with_baseline=True)
P, R, F1 = scorer.score(hyps, refs)

print(F1.mean())

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor(0.1473)


In [ ]:
# !tensorboard --logdir=runs --load_fast=true

In [ ]:
!pip install bertviz

In [ ]:
# from transformers import AutoTokenizer, AutoModel
# # tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# # model = AutoModel.from_pretrained("bert-base-uncased", output_attentions=True)
# tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])
# model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"], output_attentions=True)
# inputs = tokenizer.encode("The cat sat on the mat", return_tensors='pt')
# outputs = model(inputs)
# attention = outputs[-1]  # Output includes attention weights when output_attentions=True
# tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [ ]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view
utils.logging.set_verbosity_error()  # Suppress standard warnings

model_name = "t5-base"  # Find popular HuggingFace models here: https://huggingface.co/models
# input_text = "The cat sat on the mat"  
model = AutoModel.from_pretrained(model_name, output_attentions=True)  # Configure model to return attention values
tokenizer = AutoTokenizer.from_pretrained(model_name)
# inputs = tokenizer.encode(input_text, return_tensors='pt')  # Tokenize input text

# model = model.to(device)

# print(inputs)

In [ ]:
encoder_input_ids = tokenizer("Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding. Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. 'Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility,' chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins. TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake.", return_tensors="pt", add_special_tokens=True).input_ids
decoder_input_ids = tokenizer("summarize: TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.Time Warner's fourth quarter profits were slightly better than analysts' expectations.", return_tensors="pt", add_special_tokens=True).input_ids

In [ ]:
encoder_input_ids

tensor([[ 1980,  1085,  4888,  2900, 20055,  3199, 17610,   120,  9613,    44,
           837,   783,  6079,  2900, 21032,   687,     3, 16287,   489,  6370,
            12,  1970,     5,  2368,   115,    29,    41,     2, 19853,  6007,
            51,    61,    21,     8,   386,   767,    12,  1882,     6,    45,
         10746,  3288,    51,   215,    18,  2741,  3299,     5,    37,  1669,
             6,    84,    19,   230,    80,    13,     8,  2630,  4367,    16,
          1163,     6,     3, 26777,    45,  1085,    13,   306,    18,  9993,
          1396,  5992,    11,  1146, 24118,  1085,     5,  2900, 21032,   687,
           243,  4509,  2893,  1085,  4659,     3,  5406,    12,  1970, 11039,
           115,    29,    45,  1970, 23758,   115,    29,     5,    94,     7,
          9613,   130, 28936,    15,    26,    57,    80,    18,  1647, 11391,
            84, 13746,     3,     9,  3199, 10823,    44, 20055,  4027,     7,
             6,    11,   705,  1105,    21,    71,  

In [ ]:
decoder_input_ids

tensor([[21603,    10,  2900, 21032,   687,   243,  4509,  2893,  1085,  4659,
             3,  5406,    12,  1970, 11039,   115,    29,    45,  1970, 23758,
           115,    29,     5,  3809,     8,   423,    18,  1201,     6,  2900,
         21032,   687,  1694,     3,     9,  3199,    13,  5583,     5,  3420,
           115,    29,     6,    95,   204,  6170,    45,   165,  3888,   821,
             6,   298, 14609,     3,  4774,  4357,  5988,    12,  7514,  4416,
          4198,   115,    29,     5,  5991,  1408,    49,   120,  9613,    44,
           837,   783,  6079,  2900, 21032,   687,     3, 16287,   489,  6370,
            12,  1970,     5,  2368,   115,    29,    41,     2, 19853,  6007,
            51,    61,    21,     8,   386,   767,    12,  1882,     6,    45,
         10746,  3288,    51,   215,    18,  2741,  3299,     5,  7825,  3258,
             6,     8,   349,   243,    71,  5194,    31,     7,     3, 10067,
          3199,   274,  5916,  1173,  4659,     3,  

In [ ]:
outputs = model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)

encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0])

In [ ]:
outputs

Seq2SeqModelOutput([('last_hidden_state',
                     tensor([[[ 0.1210, -0.4335,  0.1701,  ...,  0.1708, -0.0961, -0.2530],
                              [ 0.1340, -0.4059,  0.2251,  ...,  0.2063, -0.0861, -0.3133],
                              [ 0.1116, -0.3048,  0.1068,  ..., -0.0412, -0.0365, -0.3209],
                              ...,
                              [ 0.1271, -0.4342,  0.1205,  ...,  0.1753,  0.0187, -0.1018],
                              [ 0.1558, -0.2957,  0.0597,  ...,  0.1527,  0.0226, -0.0969],
                              [ 0.1662, -0.3538,  0.1069,  ...,  0.1618,  0.0045, -0.0854]]],
                            grad_fn=<MulBackward0>)),
                    ('past_key_values',
                     ((tensor([[[[-1.4850e+00, -9.0439e-01,  1.1829e+00,  ...,  3.9449e-02,
                                  -1.3828e+00, -1.2603e+00],
                                 [ 7.2569e-01,  9.7695e-01, -1.4260e-01,  ...,  3.4269e-01,
                              

In [ ]:
from bertviz import model_view
model_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens= encoder_text,
    decoder_tokens = decoder_text
)

In [ ]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view

utils.logging.set_verbosity_error()  # Remove line to see warnings

# Initialize tokenizer and model. Be sure to set output_attentions=True.
# Load BART fine-tuned for summarization on CNN/Daily Mail dataset
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_attentions=True)

# get encoded input vectors
encoder_input_ids = tokenizer("British Airways reported pre-tax profit of £75m ($141m) for the three months to 31 December 2004 compared with £125m a year earlier. fuel costs rose by £106m or 47.3% in third quarter. but aviation analyst says surcharge revenues will still be way short of predicted extra £250m.", return_tensors="pt", add_special_tokens=True).input_ids

# create ids of encoded input vectors
decoder_input_ids = tokenizer("Rod Eddington, BA\'s chief executive, said the results were 'respectable' in a third quarter when fuel costs rose by £106m or 47.3%.To help offset the increased price of aviation", return_tensors="pt", add_special_tokens=True).input_ids

outputs = model(input_ids=encoder_input_ids, decoder_input_ids=decoder_input_ids)

encoder_text = tokenizer.convert_ids_to_tokens(encoder_input_ids[0])
decoder_text = tokenizer.convert_ids_to_tokens(decoder_input_ids[0])

model_view(
    encoder_attention=outputs.encoder_attentions,
    decoder_attention=outputs.decoder_attentions,
    cross_attention=outputs.cross_attentions,
    encoder_tokens= encoder_text,
    decoder_tokens=decoder_text
)

In [ ]:
# outputs = model(inputs)  # Run model
# attention = outputs[-1]  # Retrieve attention from model outputs
# tokens = tokenizer.convert_ids_to_tokens(inputs[0])  # Convert input ids to token strings
# model_view(attention, tokens)  # Display model view